# Set Up NVFLARE in POC Mode

[POC mode](https://nvflare.readthedocs.io/en/main/user_guide/nvflare_cli/poc_command.html) allows users to test the features of a full FLARE deployment on a single machine, without the overhead of a true distributed deployment.

Compared to the FL Simulator, where the job run is automated on a single system, POC mode allows you to establish and connect distinct server and client "systems" which can then be orchestrated using the FLARE Console.  This can be useful in preparation for a distributed deployment.

You can also experiment your deployment options (project.yml) using POC mode. 

>It is ideal to start your NVFLARE system in POC mode from a **terminal**, not from a notebook. The terminal's virual env. must match the kernel's virtual env. In our case, we are using 'nvflare_example'.

To get started, let's look at the NVFlare CLI usage for the ``poc`` subcommand:

In [ ]:

! nvflare poc -h


We can further look at the help description for each command

> We will need ```tree``` command, so we will install ```tree``` in linux, if tree is not avaialble to you, you can replace it with 'ls -al`

In [ ]:
! python -m pip install tree

#### **POC Workspace**

if you want to change default POC workspace, you can do the followings

* Config command

If you prefer not to use environment variable, you can do the followings: 

```
! nvflare config -pw /tmp/nvflare/poc

```
or 
```
! nvflare config -poc_workspace_dir /tmp/nvflare/poc
```

In [ ]:
! nvflare config -pw /tmp/nvflare/poc

Look at the nvflare config command

In [ ]:
! nvflare config -h

This should set the default variable to the following config file: ```~/.nvflare/config.conf```


One can set the POC_WORKSPACE_DIR as well as other locations such as startup_kit_dir and job_templates_dir as well. 

* Hidden Config

What config command does is to write the POC workspace directory to a hidden configuration file. 

```
~/.nvflare/config.conf
```

You can also edit directly the config file without using the config command. 

In [ ]:
!cat ~/.nvflare/config.conf

You can also env. variable, NVFLARE_POC_WORKSPACE, if you prefer. If both hidden config file and env. variable NVFLARE_POC_WORKSPACE are set, env. variable will take the precedence.
```
%env  NVFLARE_POC_WORKSPACE=/tmp/nvflare/poc
!echo $NVFLARE_POC_WORKSPACE 

```
 


### Preparing the POC workspace

Now that we've configured our POC configuration, we can prepare the POC workspace.  By default, the following command will generate POC packages for a server and two clients.
nvflare poc prepare
```
    nvflare poc prepare 
```

You can specify a different number of clients with the -n option:

```
   nvflare poc prepare -n N
```

If you running the POC prepare command, there is a prompt for user to answer.

#### Various Prepare Options

In [ ]:
!  nvflare poc prepare -h

We have many options to prepare the POC workspace. let's look at some of them

* use client's site name (instead of use default site-1,2 etc.)

```
! nvflare poc prepare -c hospital-1 hospital-2
```
This will create two clients named **hospital-1** and **hospital-2**

* -d docker_image

This instruct the provision tool to create a docker.sh use the specified docker_image. The docker.sh will start in detached mode. 

* -i project.yml 

This optional allows you experiment the your project.yml locally, try local before used in production. 

* All specified options will resulting an project.yml file located at 

<poc_workspace>/project.yml 

for example, ```/tmp/nvflare/poc/project.yml```


Let's try out some of options

* **Named Clients**

instead of using default  site-1, site-2 etc. user can specified the client names

In [ ]:
! echo 'y' | nvflare poc prepare -c hospital-1 hospital-2


Now, let's look at the result, if you have ```tree``` command installed in linux (``` pip install tree```), you can use it. otherwise, use ```ls -al```. 

In [ ]:
! tree /tmp/nvflare/poc/example_project/prod_00

Now two clients are named "hospital-1" and "hospital-2". 

* **Setup NVFLARE in docker mode**

Normally, when we using ```nvflare poc start```, the clients and server are started as process in the local host. 
What if you would like to experiment runing client and server in dockers ? You can do it now, 

```
nvflare poc prepare -n 2 -d  <docker_image>

```

This will create docker.sh for each client and server which will run docker pull docker_image from docker hub, then running the docker in detached mode. The docker_image must use nvflare docker must have flare pre-installed. For example, we use the following docker image "nvflare/nvflare"
> Note: you should build your own image if you want to include your dependencies such as pytorch etc.


In [ ]:
! echo 'y'| nvflare poc prepare -d 'nvflare/nvflare'

In [ ]:
! tree /tmp/nvflare/poc/example_project/prod_00

Notice the new "docker.sh", now each client will using docker run with we specified "nvflare/nvflare" docker image

* **simulate production deployment setup locally**

Assuming we have a new custom project.yml, we like to test out the project.yml locally before we do the real provision. 

We can actually do it with POC !


```
nvflare poc prepare -i <your project.yml>
```

Let's try this out.  we have prepared an custom_project.yml file: 


In [ ]:
!cat ./custom_project.yml

Notice in this project file, we define three sites: 

* nonprofit-server, served as Server,belong to nonprofit organizaton 
* us_hospital, served as client, belong to us-hospital
* europe-hospital, served as client, belong to eu-hospital

We also define few users: 
* admin@nonprofit.org, from nonprofit.org, serves as project admin
* admin@hospital.org.us, lead@hospital.org.us,  member@hospital.org.us from us_hospital, serves as org admin, lead and member for us_hospital respectively

* admin@hospital.org.eu, lead@hospital.org.eu,  member@hospital.org.eu from eu_hospital, serves as org admin, lead and member for eu_hospital respectively


In [ ]:
! echo 'y' | nvflare poc prepare -i ./custom_project.yml

In [ ]:
! tree /tmp/nvflare/poc/health_project/prod_00

See we actually provisioned all participants

### Prepare job example directories

By default, the command

```
nvflare poc prepare 

```
command will setup symbolic link to the <NVFLARE>/examples directory, assuming the you have use "git clone" to download the NVFLARE from github directory and you have defined the NVFLARE_HOME env. variable. 
    
What if you just want to use your own jobs or not setup the NVFLARE_HOME env. variables ? You can use the following command to setup the job directories
    
```
nvflare poc prepare-jobs-dir -j <jobs directory>

```

For example: 
    
```
nvflare poc prepare-jobs-dir -j /path/to/examples

```

    
    
    


### Start NVFlare system in POC mode

When starting the POC deployment, it's necessary to use a separate terminal since the `nvflare poc start` command will run  in the foreground emitting output from the server and any connected clients.

Also note that `nvflare poc start` starts all clients and server as well as project admin console. All other participants (org admin, lead and member) will not started. 

In many cases, we want to start the clients and server, but without the FLARE console for project admin, as we are going to interact with the system from a notebook or CLI. You first need to find out the user name for the project admin. By default, this is admin@nvidia.com

Create a terminal to execute the following command (in JupyterLab or host terminal), then

```
    nvflare poc start -ex admin@nvidia.com
```
>**Note:**
    Using ```%%bash -bg``` to run the above command in a code cell may not always work
    
#### A more complex example
    
In this example, lets see how to start the individual user participants defined in custom_project.yml. Note the project admin is "admin@nonprofit.org" instead of "admin@nvidia.com"

Please use terminal to try these commands

* We start start without project admin console

```
    nvflare poc start -ex admin@nonprofit.org
```

This will start two hospital clients and server. 

* Next we like to start US hospital's lead's FLARE console 

```

nvflare poc start -p lead@hospital.org.us

```

You are acting as US hospital's lead 






Once the system is up, we are now ready to go back to the notebook to work on your project.

### Stop FLARE system in POC

Once you are done with the FLARE system, you can shut down the poc system.
From terminal, issue the following command stop poc in the terminal
```
   nvflare poc stop
```

This will shutdown all participants including server, clients, admin consoles

If you like to shutdown invididual participant. You can use the following command

```
    nvflare poc stop -p <participant>
```

here are some examples depending on your prepared poc


```
    nvflare poc stop -p server

    nvflare poc stop -p site-1
    
    nvflare poc stop -p admin@nvidia.com
    
    nvflare poc stop -p lead@hospital.org.us
```



### Project Yaml File

Once we experimented with different options, we are ready to move on to the production beyond local host. 
You don't need to manual edit the project.yml to do so. The resulting project.yml is auto-generated for you and located at 

```
  <POC_Workspace>/project.yml
```

For example,


In [ ]:
! echo 'y' | nvflare poc prepare -c hospital_1 hospital_2 -d 'nvflare/nvflare'

In [ ]:
! tree /tmp/nvflare/poc/example_project/prod_00

In [ ]:
! cat  /tmp/nvflare/poc/project.yml

Copy this file and you are ready to go !

### Support Homomorphic Encryption (HE)

To support HE, we need the provision process to generate Tenseal homomorphic encryption context for the server and client and write them to the server and client participant folders. See [provision context](https://nvflare.readthedocs.io/en/main/programming_guide/provisioning_system.html#provision-context). This is achieved by the Provision builder, specifically for HE, HEBuilder. Instead of manually adding HEBuilder to the `project.yml` file, one can use `-he` in the POC command.

For example, if we use the above command with HE, we can write as


```
echo 'y' | nvflare poc prepare -c hospital_1 hospital_2 -d 'nvflare/nvflare' -he

```


```
    pip install nvflare[HE]
    
```


> note: Tenseal is not avaiable in Mac OS


In [ ]:
! pip install nvflare[HE]

Assuming you installed it successfully, then you can run the command

In [ ]:
! echo 'y' | nvflare poc prepare -c hospital_1 hospital_2 -d 'nvflare/nvflare' -he

In [ ]:
! cat  /tmp/nvflare/poc/project.yml


Notice:  ```path: nvflare.lighter.impl.he.HEBuilder``` is added to the YAML file.
Also note the importance of the order of the builders, as the `HEBuilder` must be configured before the `SignatureBuilder` to ensure the tenseal context files are signed.

### Clean up POC workspace

If you want to clean up the POC workspace and delete the poc workspace directory containing all packages you may have prepared, you can use the following command:

In [ ]:
! nvflare poc clean

You can check if the nvflare system is shutdown cleanly with:

```
ps -eaf | grep nvflare
```

If you see output like the following, then nvflare systems are still running:

```
510535    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-1/startup/.. -s fed_client.json --set secure_train=true uid=site-1 org=nvidia config_folder=config
510539    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-2/startup/.. -s fed_client.json --set secure_train=true uid=site-2 org=nvidia config_folder=config
510543    1932  1 18:54 pts/1    00:00:04 python3 -u -m nvflare.private.fed.app.server.server_train -m /tmp/workspace/example_project/prod_00/localhost/startup/.. -s fed_server.json --set secure_train=true org=nvidia config_folder=config
```

If you have already used ``nvflare poc clean`` to delete the workspace, you may need to kill the processes manually.
